In [1]:
# Add additional library
import sys, os
data_lib = os.path.abspath('../data')
if data_lib not in sys.path:
    sys.path.append(data_lib)

from data_utils import get_data_path_manager, get_dataset_path
import pickle
import numpy as np
from pathlib import Path
from tqdm import tqdm
import json

In [2]:
BVP_SAMPLING_RATE = 64
EDA_SAMPLING_RATE = 4

dp_manager = get_data_path_manager()

In [3]:
def resampling_indices(indices, target_sr, original_sr):
    rate = target_sr / original_sr
    resampled_indices = [int(round(ind) * rate) for ind in indices]
    return resampled_indices

In [4]:
def assert_resampling_indices(fst_inds, fnd_inds, fsr, sst_inds, snd_inds, ssr):
    assert(len(fst_inds) == len(sst_inds))
    assert(len(fnd_inds) == len(fnd_inds))

    num_indices = len(fst_inds)
    for i in range(num_indices):
        f_len = round((fnd_inds[i] - fst_inds[i]) / fsr)
        s_len = round((snd_inds[i] - sst_inds[i]) / ssr)
        assert(f_len == s_len)

---

# WESAD CURSOR-INDICES PREPROCESSING

In [5]:
WESAD_LABELS = ['', 'baseline', 'stress', 'amusement', 'meditation']
ORIGINAL_SAMPLING_RATE = 700
metadata_file_path = str(Path(dp_manager.dataset_path) / 'wesad_wrist_metadata.json')

wesad_metadata = {}
user_ids = sorted(os.listdir(dp_manager.WESAD_dataset_path))
for user_id in tqdm(user_ids):
    data_path = str(Path(dp_manager.WESAD_dataset_path) / user_id / f'{user_id}.pkl')
    data = pickle.load(open(data_path, 'rb'), encoding = 'bytes')

    labels = data[b'label'].tolist()
    labels.insert(-1, 0)
    num_labels = len(labels)
    starting_indices = []
    ending_indices = []
    gt = []
    for i in range(1, num_labels):
        if labels[i] != labels[i-1]:
            if 0 < labels[i] < 5: 
                starting_indices.append(i)
                gt.append(WESAD_LABELS[labels[i]])
            if 0 < labels[i-1] < 5: ending_indices.append(i-1)
    bvp_starting_indices = resampling_indices(starting_indices, BVP_SAMPLING_RATE, ORIGINAL_SAMPLING_RATE)
    bvp_ending_indices = resampling_indices(ending_indices, BVP_SAMPLING_RATE, ORIGINAL_SAMPLING_RATE)
    assert_resampling_indices(starting_indices, ending_indices, ORIGINAL_SAMPLING_RATE, bvp_starting_indices, bvp_ending_indices, BVP_SAMPLING_RATE) 

    eda_starting_indices = resampling_indices(starting_indices, EDA_SAMPLING_RATE, ORIGINAL_SAMPLING_RATE)
    eda_ending_indices = resampling_indices(ending_indices, EDA_SAMPLING_RATE, ORIGINAL_SAMPLING_RATE)
    assert_resampling_indices(starting_indices, ending_indices, ORIGINAL_SAMPLING_RATE, eda_starting_indices, eda_ending_indices, EDA_SAMPLING_RATE)

    wesad_metadata[user_id] = {
        'labels': gt,
        'bvp': {
            'starting_indices': bvp_starting_indices,
            'ending_indices': bvp_ending_indices
        },
        'eda': {
            'starting_indices': eda_starting_indices,
            'ending_indices': eda_ending_indices
        }
    }

with open(metadata_file_path, 'w') as f:
    json.dump(wesad_metadata, f, indent = 4)

100%|██████████| 15/15 [00:32<00:00,  2.14s/it]
